In [1]:
import pandas as pd
import numpy as np
import pickle
import sys
sys.path.append("/home/sano/TCGA_GSCE/TCGA_RF_Version_Last/")

import Mut_pred_V3 as Mut_pred
from sklearn.ensemble import RandomForestClassifier
from collections import Counter

In [2]:
import importlib
importlib.reload(Mut_pred)

<module 'Mut_pred_V3' from '/home/sano/TCGA_GSCE/TCGA_RF_Version_Last/Mut_pred_V3.py'>

In [3]:
Df=pd.read_csv("/data2/sano/TCGA_Version_LAST/TCGA_RNA_common_patient.csv", index_col=0)
Df=Df.T
Driver_Mutation_Matrix=pd.read_csv("/data2/sano/TCGA_Version_LAST/TCGA_Answer_common_patient.csv", index_col=0)
gene_id=pd.read_table("/data2/sano/TCGA_Version_LAST/Gene_id.txt", sep="\t", index_col=0)
Gene_N=pd.read_csv( "/data2/sano/TCGA_Version_LAST/Biogrid_Gene_100_gene.csv", index_col=0)

In [4]:
filename = "/data2/sano/TCGA_Version_LAST/Train_Test_patient.pickle"
with open(filename, 'rb') as f:
    Train_Test_Patients=pickle.load(f)

# Training

In [ ]:
for GENE in Gene_N.columns[0:25]:    
    gene_100=Gene_N[GENE]
    gene_100= [x for x in Gene_N[GENE]  if x == x]
    TCGA_MTX=Mut_pred.MTX(Df, gene_100)
    TCGA_MTX=TCGA_MTX.T
    filename = "/data2/sano/TCGA_Version_LAST/Models/MTX_"+GENE+".csv"
    TCGA_MTX.to_csv(filename )
        
    train_x =  TCGA_MTX.loc[Train_Test_Patients["train"]]
    train_y = Driver_Mutation_Matrix[GENE].loc[Train_Test_Patients["train"]]

    clf1 = RandomForestClassifier(max_depth=5, n_estimators=10000, random_state=0, class_weight='balanced')
    clf1.fit(train_x, train_y)

    filename = "/data2/sano/TCGA_Version_LAST/Models/Model_"+GENE+".sav"
    with open(filename, 'wb') as f:
        pickle.dump(clf1, f)